<a href="https://colab.research.google.com/github/Vakhranev/Pushkina/blob/main/%D0%9C%D1%91%D1%80%D0%B4%D0%B6%20%D1%81%20TF-IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Загружаем файл
df = pd.read_csv("обновленная_лексика_мигранты.csv")

# Создаём новый столбец "words", извлекая первую часть перед "__"
df["words"] = df["lemma__pos"].str.split("__").str[0]

# Сохраняем обновлённый файл
df.to_csv("обновленная_лексика_мигранты_обработанная.csv", index=False)

print("Файл обновленная_лексика_мигранты_обработанная.csv сохранён.")

Файл обновленная_лексика_мигранты_обработанная.csv сохранён.


In [2]:
import pandas as pd

# Загружаем файлы
final_merged = pd.read_excel("final_merged (1).xlsx")
migrant_words = pd.read_csv("обновленная_лексика_мигранты_обработанная.csv")

# Пробуем объединить по "Начальные формы"
merged_df = final_merged.merge(migrant_words, left_on="Начальные формы", right_on="words", how="outer")

# Если не получилось, пробуем по "termin"
if merged_df.empty:
    merged_df = final_merged.merge(migrant_words, left_on="termin", right_on="words", how="outer")

# Если всё ещё не получилось, пробуем по "Термины глоссариев"
if merged_df.empty:
    merged_df = final_merged.merge(migrant_words, left_on="Термины глоссариев", right_on="words", how="outer")

# Сохраняем результат
merged_df.to_excel("final_merged_updated.xlsx", index=False)

print("Файл final_merged_updated.xlsx сохранён.")

Файл final_merged_updated.xlsx сохранён.
